# Agency Prefix Code Profiles
Which agencies apply the most or have the most prefix codes? Which ones have the least? 

Data: 
* [Oblgiated list](https://dot.ca.gov/programs/local-assistance/reports/e-76-obligated)

* running notebook with updated locode data 'obligated_list_1202.csv'

In [7]:
import numpy as np
import pandas as pd
from siuba import *

import altair as alt
import altair_saver
from plotnine import *

from shared_utils import altair_utils
alt.themes.enable("fivethirtyeight")
from calitp import *

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Markdown
from IPython.core.display import display

import _clean_data
import _dla_utils

In [2]:
#df = _clean_data.make_clean_data()
df= pd.read_parquet("dla_df.parquet")

In [3]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,...,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO,5904,2018.0,Humboldt County
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,...,nan,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,32D0,2018.0,Mendocino County
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,...,nan,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,4820,2018.0,Humboldt County
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,...,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,5924,2018.0,Sacramento County
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,...,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,5924,2018.0,Sacramento County


In [4]:
#making sure the locode and agency updated in the data.
## if updated, there will only be one agency listed.
## if multiple agencies listed, go back to add_locode.ipynb and run that notebook

df>>filter(_.locode=='6000')>>count(_.primary_agency_name)

,primary_agency_name,n
0,San Francisco Bay Area Rapid Transit District,41


## How many unique prefix codes per agency

### Top Agencies
finding the top agencies with the most unique prefix codes

In [9]:
top = _dla_utils.get_nunique(df, 'prefix', 'primary_agency_name')

In [10]:
top

,primary_agency_name,n
276,Los Angeles,34
277,Los Angeles County,31
434,San Bernardino Associated Governments,29
71,Caltrans,27
534,Stockton,23
...,...,...
591,Weed,1
595,Western Shasta Resource Conservation District,1
602,Willits,1
603,Willows,1


In [13]:
_dla_utils.interactive_widget_counts(df,  'primary_agency_name', 'prefix')

Dropdown(description='Primary_Agency_Name', options=('Access Services', 'Agoura Hills', 'Alameda', 'Alameda - …

Output()

#### Printing top 5 programs for the top agencies


In [14]:
top

,primary_agency_name,n
276,Los Angeles,34
277,Los Angeles County,31
434,San Bernardino Associated Governments,29
71,Caltrans,27
534,Stockton,23
...,...,...
591,Weed,1
595,Western Shasta Resource Conservation District,1
602,Willits,1
603,Willows,1


In [15]:
def find_agencies_top(agency=top.primary_agency_name.unique().tolist()):
    
    for agency in top.primary_agency_name.unique().tolist(): 
            count_top = ((df>>filter(_.primary_agency_name==agency)>> count(_.prefix)>>arrange(-_.n)).head(5)>>select(_.prefix))
            
            display(Markdown(f"**Top Prefix Codes {agency} uses:**"))
            display(count_top)
            
    return

In [16]:
agency=top.primary_agency_name.unique().tolist()
find_agencies_top(agency)

**Top Prefix Codes Los Angeles uses:**

,prefix
26,STPL
20,HSIPL
10,CML
2,ATPL
5,BHLS


**Top Prefix Codes Los Angeles County uses:**

,prefix
12,ER
18,HSIPL
1,ACSTP
4,BPMPL
23,STPL


**Top Prefix Codes San Bernardino Associated Governments uses:**

...

,prefix
0,BRLS


**Top Prefix Codes Willows uses:**

,prefix
0,RPSTPL


**Top Prefix Codes Yountville uses:**

,prefix
0,RPSTPL


Popular codes in top agencies:
   * STP/STLN: Surface Transportation Program/Surface Transportation Program Local National Highway System
   * HSIP: Highway Safety Improvement Program 
   * CML: Congestion Mitigation & Air Quality Improvement Program 
   * BHLS: ?
   * BRLO: Highway Bridge Program (HBP) - Off Federal-aid System 

### Bottom Agencies

In [17]:
(df 
     >> group_by(_.primary_agency_name) 
     >> summarize(n=_.prefix.nunique())
     >> filter(_.n<=1)
 
)

,primary_agency_name,n
4,Alameda Corridor Transportation Authority,1
8,Alameda County Transportation Improvement Auth...,1
9,Alameda County Waste Management Authority,1
13,Amador City,1
15,Amador County Transportation Commission,1
...,...,...
591,Weed,1
595,Western Shasta Resource Conservation District,1
602,Willits,1
603,Willows,1


In [18]:
df >> group_by(_.primary_agency_name) >> summarize(n=_.prefix.nunique()) >> arrange(_.n)>>filter(_.n<=3)

,primary_agency_name,n
4,Alameda Corridor Transportation Authority,1
8,Alameda County Transportation Improvement Auth...,1
9,Alameda County Waste Management Authority,1
13,Amador City,1
15,Amador County Transportation Commission,1
...,...,...
587,Walnut Creek,3
592,West Covina,3
597,Westminster,3
605,Winters,3


In [19]:
bottom = df >> group_by(_.primary_agency_name) >> summarize(n=_.prefix.nunique()) >> arrange(_.n)>>filter(_.n<=3)

In [20]:
bottom.sample(9)

,primary_agency_name,n
387,Plymouth,2
528,SouthWest Transportation Agency,2
460,San Joaquin Regional Transit District,2
355,Orange Cove,2
336,Natomas Unified School District,1
405,Redondo Beach,2
107,Compton,2
522,South Coast Area Transit,1
307,Millbrae,2


In [21]:
agency=bottom.primary_agency_name.unique().tolist()


def find_agencies_bottom(agency=bottom.primary_agency_name.unique().tolist()):
    
    for agency in bottom.primary_agency_name.unique().tolist(): 
            count_bottom = ((df>>filter(_.primary_agency_name==agency)>> count(_.prefix)>>arrange(_.n)).head(5)>>select(_.prefix))
            
            display(Markdown(f"**Top Prefix Codes {agency} uses:**"))
            display(count_bottom)
            
    return

In [22]:
print(len(agency))

302


In [23]:
find_agencies_bottom(agency)

**Top Prefix Codes Alameda Corridor Transportation Authority uses:**

,prefix
0,HPLUL


**Top Prefix Codes Alameda County Transportation Improvement Authority uses:**

,prefix
0,STPL


**Top Prefix Codes Alameda County Waste Management Authority uses:**

...

,prefix
2,STPL
0,ATPL
1,HSIPL


**Top Prefix Codes Winters uses:**

,prefix
1,HPLUL
2,STPL
0,BRLS


**Top Prefix Codes Yreka City uses:**

,prefix
2,BRLS
0,ACSTP
1,BPMP


Popular codes in bottom agencies: 
   * FTACML
   * STP/RPSTP
   * HSIP
   * CML 
   * BRLS
   * BRLO
   * CMLNI
    

* 140 agencies with 1 unique prefix code.
* 339 agencies with 3 or less prefix codes. 

## In which Caltrans Districts do these agencies have the most obligations?

### For the agencies in districts that applied the least

* joining the bottom agencies to the full df
* grouping by dsitrict to count the number of obligation occurances 

In [24]:
bottom = (df 
     >> group_by(_.primary_agency_name) 
     >> summarize(n=_.prefix.nunique())
     >> filter(_.n<=3)
)

In [25]:
bottom_join = inner_join(df, bottom, on = "primary_agency_name")

In [26]:
bottom_dist = bottom_join >> group_by(_.dist) >> count(_.dist) >> arrange(-_.n)

In [27]:
bottom_dist.rename(columns={'n': 'bottom_n'}, inplace=True)

In [28]:
bottom_dist

,dist,bottom_n
3,4,394
6,7,382
9,10,352
5,6,264
2,3,259
7,8,209
10,11,129
11,12,114
4,5,81
0,1,60


### For the agencies in districts that applied the most

* joining the top agencies to the full df
* grouping by dsitrict to count the number of obligation occurances 

In [29]:
top_join = inner_join(df, top, on = "primary_agency_name")

In [30]:
top_dist = top_join >> group_by(_.dist) >> count(_.dist) >> arrange(-_.n)

In [31]:
top_dist.rename(columns={'n': 'top_n'}, inplace=True)

In [32]:
top_dist

,dist,top_n
4,4,3247
6,6,3084
7,7,2785
3,3,2517
10,10,2203
5,5,1337
8,8,1309
1,1,1161
11,11,920
12,12,666


### Number of Obligations for Top and Bottom on District
 
* joining the dataframes we just made to compare the top and bottom easier
* finding which districts have the most obligations 

In [33]:
dist = full_join(bottom_dist, top_dist, on = "dist")

In [34]:
dist>>arrange(_.dist)

,dist,bottom_n,top_n
12,0,NaN,4
9,1,60.0,1161
10,2,52.0,614
4,3,259.0,2517
0,4,394.0,3247
8,5,81.0,1337
3,6,264.0,3084
1,7,382.0,2785
5,8,209.0,1309
11,9,39.0,267


In [35]:
dist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   dist      13 non-null     int64  
 1   bottom_n  12 non-null     float64
 2   top_n     13 non-null     int64  
dtypes: float64(1), int64(2)
memory usage: 416.0 bytes


In [36]:
dist['bottom_n'] = dist['bottom_n'].fillna(0)

In [37]:
dist['bottom_n'] = dist['bottom_n'].astype(int)

In [38]:
dist.rename(columns={'dist': 'district'}, inplace=True)

In [39]:
dist>>arrange(_.district)

,district,bottom_n,top_n
12,0,0,4
9,1,60,1161
10,2,52,614
4,3,259,2517
0,4,394,3247
8,5,81,1337
3,6,264,3084
1,7,382,2785
5,8,209,1309
11,9,39,267


## Findings:



* **339 agencies** have **3 or less** unique prefixes. 
* **80 agencies** have **10 or more** unique prefixes, with the following **9 agenceies** listed with **20 or more**. 
* **District 7 and District 2** have the most obligations out of the top and bottom agencies 



| Top Agencies  | Bottom Agencies (sample of 9)|
| ------------- | ------------- | 
| Los Angeles | Sonoma County Transportation Authority | 
| Caltrans  |Port Of Long Beach |   
| Los Angeles County |Newport Beach |    
| San Bernardino Associated Governments | Portola | 
| San Diego | Calaveras Council Of Governments | 
| Riverside County | Hawthorne | 
| San Luis Obispo County | Corcoran |  
| Stockton | Claremont | 
| El Dorado County | El Cajon |  


